In [ ]:
from hpelm import ELM
import numpy as np
import time
from sklearn.utils import shuffle

In [ ]:
# from PIL import Image
# from numpy import array
# from scipy.misc import imresize

In [ ]:
# from os import listdir
# from os.path import isfile, join
# from cutImages import cut_image


# def read_images(path):
#     fullList = []
#     for c in range(1, 100):
#         mypath = path + str(c) + "/"
#         onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
#         print (len(onlyfiles), c)
#         for x in onlyfiles:
#             if x == 'bb_info.txt':
#                 continue
#             img = cut_image(mypath + x)
#             arr = array(img)
#             arr = imresize(arr, [32, 32, 3])
#             r = arr[:, :, 0].flatten()
#             g = arr[:, :, 1].flatten()
#             b = arr[:, :, 2].flatten()
#             label = [c]
#             out = np.array(list(label) + list(r) + list(g) + list(b), np.int)
#             fullList.append(out)
#     shuffle(fullList)
#     imagesList = np.array(fullList, dtype=int)
#     print ("images shape: {}".format(imagesList.shape))
#     return imagesList

In [ ]:
# imagesList = np.load('UECFOOD_100_1000Features_train.npy')
trainingdata = np.loadtxt('/home/arl/aymen/Food-Dataset/food10/1664features-cnn-finetuned.svm')

In [ ]:
trainingdata.shape

from cPickle import load
all_features_file = '/media/aymen/DATA/datasets/ghalib-features/data_features_orb_UECFOOD100.pickle'
all_features = load(open(all_features_file, 'rb'))

img_class = all_features[0][0].split('/')[-2]
print(img_class)

fet = all_features[0][1]
print fet.shape

count = 0 
for i, val in all_features:
    if count < 3 :
        img_class = i.split('/')[-2]
        print img_class, val
        count +=1
    else:
        break
        

In [ ]:
imagesList = shuffle(trainingdata)
# index = np.argwhere(imagesList== 175.0)
# imagesList = np.delete(imagesList, index,axis=0)
precentage = 0.75
print ('imaglist full', imagesList.shape)
trainImagesList = imagesList[0:int(imagesList.__len__() * precentage), :]
testImagesList = imagesList[int(imagesList.__len__() * precentage):, :]
print ('trainImagesList', trainImagesList.shape)
print ('testImagesList', testImagesList.shape)
print (trainImagesList.shape[0] + testImagesList.shape[0])
print(imagesList[3])
print(np.unique(imagesList[:,0]))

In [ ]:
from collections import Counter
Counter(imagesList[:,0])

Counter({0.0: 620,
         1.0: 136,
         2.0: 104,
         3.0: 112,
         4.0: 218,
         5.0: 120,
         6.0: 107,
         7.0: 100,
         8.0: 148,
         9.0: 233,
         10.0: 134,
         11.0: 163,
         12.0: 130,
         13.0: 152,
         14.0: 106,
         15.0: 163,
         16.0: 353,
         17.0: 139,
         18.0: 112,
         19.0: 131,
         20.0: 151,
         21.0: 137,
         22.0: 134,
         23.0: 115,
         24.0: 115,
         25.0: 120,
         26.0: 126,
         27.0: 101,
         28.0: 102,
         29.0: 115,
         30.0: 728,
         31.0: 113,
         32.0: 118,
         33.0: 112,
         34.0: 121,
         35.0: 107,
         36.0: 113,
         37.0: 167,
         38.0: 106,
         39.0: 105,
         40.0: 121,
         41.0: 116,
         42.0: 104,
         43.0: 124,
         44.0: 181,
         45.0: 150,
         46.0: 122,
         47.0: 105,
         48.0: 102,
         49.0: 108,
         50.0: 118,
         51.0: 154,
         52.0: 140,
         53.0: 102,
         54.0: 109,
         55.0: 116,
         56.0: 246,
         57.0: 135,
         58.0: 108,
         59.0: 110,
         60.0: 117,
         61.0: 120,
         62.0: 111,
         63.0: 107,
         64.0: 131,
         65.0: 224,
         66.0: 147,
         67.0: 153,
         68.0: 157,
         69.0: 109,
         70.0: 117,
         71.0: 107,
         72.0: 109,
         73.0: 105,
         74.0: 147,
         75.0: 111,
         76.0: 122,
         77.0: 118,
         78.0: 100,
         79.0: 110,
         80.0: 115,
         81.0: 126,
         82.0: 142,
         83.0: 125,
         84.0: 115,
         85.0: 128,
         86.0: 342,
         87.0: 109,
         88.0: 115,
         89.0: 169,
         90.0: 117,
         91.0: 165,
         92.0: 167,
         93.0: 111,
         94.0: 108,
         95.0: 105,
         96.0: 126,
         97.0: 102,
         98.0: 153,
         99.0: 138})

In [ ]:
# images_norm = imagesList[:,1:]
# labels = imagesList[:,0]
# from sklearn.preprocessing import normalize
# images_norm = normalize(images_norm)

# print (images_norm.shape)
# print('labels',labels.shape)
# print(images_norm[324])
# print(np.unique(images_norm))
# trainImagesList = images_norm[0:int(images_norm.__len__() * .8), :]
# testImagesList = images_norm[int(images_norm.__len__() * .8):, :]

# trainImagesList_labels = labels[0:int(labels.__len__() * .8)]
# testImagesList_labels = labels[int(labels.__len__() * .8):]

# print ('trainImagesList', trainImagesList.shape)
# print ('testImagesList', testImagesList.shape)
# print ('trainImagesList_labels', trainImagesList_labels.shape)
# print ('testImagesList_labels', testImagesList_labels.shape)
# # images = norm2


In [ ]:
epoch = 1
batch_size = trainImagesList.__len__() / epoch
hidden_num = 2500
_inputs = 1664
_outputs = 10
elm = ELM(_inputs, _outputs, batch=batch_size)
elm.add_neurons(hidden_num, "tanh")
print("batch_size : {}".format(batch_size))
print("hidden_num : {}".format(hidden_num))

In [ ]:
k=1
old_batch_size_k = 0
data = trainImagesList
start_time = time.time()
while k <= epoch:
    # shuffle(trainImagesList)
    print("batch : {}".format(k))
    # print (data.shape, data.dtype)
    # dynamic
#     train_x = np.array(data[old_batch_size_k:(batch_size * k), 1:], dtype="float")
#     train_y = np.array(data[old_batch_size_k:(batch_size * k), 0], dtype="int")
#     if k == epoch:
#         train_x = np.array(data[old_batch_size_k:, 1:], dtype="float")
#         train_y = np.array(data[old_batch_size_k:, 0], dtype="int")
#     old_batch_size_k = batch_size * k

    # whole
    train_x = np.array(data[:, 1:], dtype="float")
    train_y = np.array(data[:, 0], dtype="int")
    #separate labels and images 
#     train_x = np.array(trainImagesList, dtype="float")
#     train_y = np.array(trainImagesList_labels, dtype="int")
    
    print ("X", train_x.shape)
    print ("Y", train_y.shape)
    # end whole
    train_y = np.eye(np.max(train_y) + 1)[train_y]
    elm.train(train_x, train_y)
    k += 1

end_time = time.time()
print("%s seconds" % (end_time - start_time))

In [ ]:
def predict(_testImagesList):
    # testImagesList = np.load(testPath)
    testList = _testImagesList

    # print ('testImagesList', testList, testList.shape)
    test_x = np.array(testList[:, 1:], dtype="float")
    test_y = np.array(testList[:, 0], dtype="int")
    test_y = np.eye(np.max(test_y) + 1)[test_y]
    #
    Y = elm.predict(test_x)
    predict = []
    for y in Y:
        predict.append(y.argmax())

    gt = []
    for y in test_y:
        gt.append(y.argmax())

    save = []
    for _ in np.arange(0, len(gt)):
        k = True if gt[_] == predict[_] else False
        save.append(k)

    return save

In [ ]:
# def predict_labels(_testImagesList,_labels):
#     # testImagesList = np.load(testPath)
#     testList = _testImagesList
#     labels = _labels

#     # print ('testImagesList', testList, testList.shape)
#     test_x = np.array(testList, dtype="float")
#     test_y = np.array(labels, dtype="int")
#     test_y = np.eye(np.max(test_y) + 1)[test_y]
#     #
#     Y = elm.predict(test_x)
#     predict = []
#     for y in Y:
#         predict.append(y.argmax())

#     gt = []
#     for y in test_y:
#         gt.append(y.argmax())

#     save = []
#     for _ in np.arange(0, len(gt)):
#         k = True if gt[_] == predict[_] else False
#         save.append(k)

#     return save

In [ ]:
test = predict(testImagesList)
print("test accuracy: {0}".format(np.mean(test)))
train = predict(trainImagesList)
print("train accuracy: {0}".format(np.mean(train)))

In [ ]:
# test = predict_labels(testImagesList,testImagesList_labels)
# print("test accuracy: {0}".format(np.mean(test)))
# train = predict_labels(trainImagesList,trainImagesList_labels)
# print("train accuracy: {0}".format(np.mean(train)))

In [ ]:
test accuracy: 0.867333333333
train accuracy: 0.992444444444